In [1]:
import arcpy
from arcpy.sa import *
import datetime

In [7]:
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\National_report_cards\\National_report_cards.gdb"

In [8]:
arcpy.ListRasters()

['Antro4classes',
 'gHM_Irrigated',
 'gHM_Rainfed',
 'gHM_Rangeland',
 'gHM_Urban',
 'amph_rar',
 'bird_rar',
 'spi_raster_1980',
 'spi_10_rescale_CopyRaster',
 'CopyRaster_OutRasterdataset_spi_3_rescale',
 'CopyRaster_OutRasterdataset_spi_4_rescale',
 'CopyRaster_OutRasterdataset_spi_5_rescale',
 'CopyRaster_OutRasterdataset_spi_6_rescale',
 'CopyRaster_OutRasterdataset_spi_7_rescale',
 'CopyRaster_OutRasterdataset_spi_8_rescale',
 'CopyRaster_OutRasterdataset_spi_9_rescale',
 'CopyRaster_OutRasterdataset_spi_10_rescale',
 'ghm_reclass']

In [9]:
ghm_reclass = "ghm_reclass"

In [11]:
reclass_dict = {
    "ghm_0": "1",
    "ghm_low": "2",
    "ghm_moderate": "3",
    "ghm_high": "4",
    "ghm_vhigh": "5"
}

In [12]:
for i, j in reclass_dict.items():
    start = datetime.datetime.now()
    out_name = f"C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\National_report_cards\\National_report_cards.gdb\\gHM_{i}"
    outSetNull = SetNull(ghm_reclass, ghm_reclass, f"VALUE <> {j}")
    outSetNull.save(out_name)  #this does not use the env parameter, it uses the location of the notebook as root
    end = datetime.datetime.now() - start
    print(f"{i} finished in {end}")

ghm_0 finished in 0:01:25.108831
ghm_low finished in 0:01:41.533444
ghm_moderate finished in 0:01:48.440566
ghm_high finished in 0:01:40.451588
ghm_vhigh finished in 0:01:24.092308


In [21]:
ghm_class_rasters = arcpy.ListRasters("gHM_ghm*")
for raster in ghm_class_rasters:
    print(raster)

gHM_ghm_0
gHM_ghm_low
gHM_ghm_moderate
gHM_ghm_high
gHM_ghm_vhigh


In [17]:
gadm_shp = "gadm_prop_sentence_cea"

In [22]:
ghm_class_rasters = ghm_class_rasters[1:5]

In [23]:
ghm_class_rasters

['gHM_ghm_low', 'gHM_ghm_moderate', 'gHM_ghm_high', 'gHM_ghm_vhigh']

In [24]:
for raster in ghm_class_rasters:
    ghm_class=raster.split(f"_")[2]
    outTableName = f"ZSat_{ghm_class}"
    start = datetime.datetime.now()
    outZSat = ZonalStatisticsAsTable(gadm_shp,
                                     "GID_0", 
                                     raster,
                                     outTableName, "DATA", "ALL")
    end = datetime.datetime.now() - start
    print(f"{ghm_class} finished in {end}")

low finished in 0:08:24.956788
moderate finished in 0:08:57.677401
high finished in 0:08:08.266309
vhigh finished in 0:08:08.978885


In [25]:
zsa_tables = arcpy.ListTables("ZSat_*")

In [28]:
zsa_tables

['ZSat_0', 'ZSat_low', 'ZSat_moderate', 'ZSat_high', 'ZSat_vhigh']

In [27]:
zsa_tables = zsa_tables[4:9]

In [30]:
ZSaTGrid = "ZonalSt_gadm_ghm"

In [34]:
join_field_name = "GID_0"

In [33]:
grid_fields = arcpy.ListFields(ZSaTGrid)
for fl in grid_fields:
    print(fl.name)

OBJECTID
GID_0
ZONE_CODE
COUNT
AREA
MIN
MAX
RANGE
MEAN
STD
SUM
VARIETY
MAJORITY
MINORITY
MEDIAN


In [36]:
for tbl in zsa_tables:
    pos = len(arcpy.ListFields(ZSaTGrid))
    start = datetime.datetime.now()
    arcpy.management.JoinField(ZSaTGrid, join_field_name, tbl, join_field_name, "COUNT")
    ghm_class=tbl.split("_")[1]
    newField_name = f"COUNT_{ghm_class}" 
    oldField_name = arcpy.ListFields(ZSaTGrid)[pos].name 
    arcpy.AlterField_management(ZSaTGrid, oldField_name, newField_name, newField_name)
    end = datetime.datetime.now() - start
    print(f"{tbl} finished in {end}")

ZSat_0 finished in 0:00:02.843436
ZSat_low finished in 0:00:02.739634
ZSat_moderate finished in 0:00:02.992631
ZSat_high finished in 0:00:02.769888
ZSat_vhigh finished in 0:00:02.847148


In [37]:
for tbl in zsa_tables:
    ghm_class = tbl.split("_")[1]
    prop_name = f"prop_{ghm_class}"
    start = datetime.datetime.now()
    arcpy.management.AddField(ZSaTGrid, 
                          prop_name, 
                          "DOUBLE", 
                          None, None, None, 
                          prop_name, "NULLABLE", "NON_REQUIRED", '')
    COUNT_name = f"COUNT_{ghm_class}" 
    py_formula = f"(!{COUNT_name}! / !COUNT!) * 100"
    arcpy.management.CalculateField(ZSaTGrid, 
                                prop_name, 
                                py_formula, 
                                "PYTHON3", '') 
    end = datetime.datetime.now() - start
    print(f"{ghm_class} finished in {end}")

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

...

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

vhigh finished in 0:00:02.512094


In [39]:
fieldObs = arcpy.ListFields(ZSaTGrid)  
fieldNames = []
for field in fieldObs:
    fieldNames.append(field.name)
del fieldObs
fieldCount = len(fieldNames)

In [40]:
start = datetime.datetime.now()
with arcpy.da.UpdateCursor(ZSaTGrid, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)
del curU
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:00.154011
